In [1]:
import io
import json
import os
import random
import subprocess
import time
from collections import defaultdict
from datetime import datetime
from pathlib import Path

import numpy
import pytesseract
from IPython.display import clear_output, display
from PIL import Image, ImageDraw, ImageEnhance, ImageFilter
from tqdm import tqdm

In [2]:
from matplotlib import pyplot as plt

In [3]:
def shell(cmd, debug=False, returns_POpen=False, close_fds=False):
    if debug:
        print(cmd)

    if returns_POpen:
        process = subprocess.Popen(cmd, close_fds=close_fds)
        return process

    process = subprocess.run(cmd, stdout=subprocess.PIPE)
    return process.stdout.decode("utf-8").strip().splitlines()

In [4]:
def adb(cmd, debug=False, returns_POpen=False, close_fds=False):
    # This function runs adb commands on your connected device or emulator.
    if type(cmd) == str:
        cmd = cmd.split(" ")
    cmd = ["adb"] + cmd
    return shell(cmd, debug=debug, returns_POpen=returns_POpen, close_fds=close_fds)


adb("wait-for-device")

[]

In [5]:
def pullPhoneScreen(resize_ratio=None, as_numpy=False, print_times=False):
    s = time.time()
    adb("shell screencap -p /sdcard/screen.png")
    adb("pull /sdcard/screen.png ./game.png")
    adb("shell rm /sdcard/screen.png")
    im = Image.open("game.png")
    im = im.convert("RGB")
    if resize_ratio is not None:
        im = im.resize(
            (int(im.width * resize_ratio), int(im.height * resize_ratio)),
            Image.Resampling.LANCZOS,
        )
    if print_times:
        print("pull image took ", time.time() - s)
    if as_numpy:
        return numpy.array(im)
    return im

In [6]:
def PillowToCv2(img):
    img = np.array(img)
    im = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return im

In [7]:
import cv2
import numpy as np

In [8]:
class Beholder_Layer:
    def __init__(self, name, data, offsets=None):
        self.offsets = offsets
        self.name = name
        self.data = data

    def show(self):
        img = cv2.cvtColor(self.data, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(img))

    def chop(self, new_name, x, y, h, w):
        return Beholder_Layer(
            name=new_name, data=self.data[y : y + h, x : x + w], offsets=(x, y)
        )

In [9]:
class Beholder_Layer_Chopper:
    def __init__(self, name):
        self.name = name

    def run(self, bh):
        pass

In [10]:
class Beholder_Matcher:
    def __init__(self, name, layer, data):
        self.name = name
        self.layer = layer
        self.data = data

    def show(self):
        img = cv2.cvtColor(self.data, cv2.COLOR_BGR2RGB)
        display(Image.fromarray(img))

    def __repr__(self):
        return f"BW: {self.name}"

In [11]:
class Beholder:
    def __init__(self, videoFrameGenerator):
        self.generator = videoFrameGenerator
        self.matchers = {}
        self.threshhold = 0.95
        self.layers = {}
        self.layer_modifiers = []

    def addLayerModifer(self, ch: Beholder_Layer_Chopper):
        self.layer_modifiers.append(ch)

    def addDefaultMatcher(self, name, filename, layer="gray", convertToGray=True):
        if Path(filename).exists():

            img = PillowToCv2(Image.open(filename))
            if convertToGray:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            self.matchers[name] = Beholder_Matcher(name, layer, img)
        else:
            print("File is missing")

    def readNextImage(self):
        self.layers["image"] = Beholder_Layer(
            name="image", data=PillowToCv2(self.generator())
        )

    def digestImage(self):
        for m in self.layer_modifiers:
            try:
                self.layers.update(m.run(self))
            except Exception as e:
                print(e)

    def findMatches(self, matchers=None):
        if matchers is None:
            matchers = self.matchers
        self.readNextImage()
        self.digestImage()
        matches = defaultdict(list)
        for m in matchers:
            m = matchers[m]

            result = cv2.matchTemplate(
                self.layers[m.layer].data, m.data, method=cv2.TM_CCOEFF_NORMED
            )
            min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)

            if max_val > self.threshhold:
                height, width = m.data.shape[:2]
                layer_offset_y = 0
                layer_offset_x = 0
                if self.layers[m.layer].offsets is not None:
                    layer_offset_x, layer_offset_y = self.layers[m.layer].offsets
                top_left = max_loc
                center = (
                    (layer_offset_x + top_left[0] + (width / 2)),
                    (layer_offset_y + top_left[1] + (height / 2)),
                )

                threshold = 0.8
                loc = np.where(result >= threshold)

                f = set()
                sensitivity = 100
                for pt in zip(*loc[::-1]):
                    f.add(
                        (
                            round(layer_offset_x + pt[0] / sensitivity),
                            layer_offset_y + round(pt[1] / sensitivity),
                        )
                    )

                found_count = len(f)
                matches[m.name].append((m, center, found_count))

        return dict(matches)

In [12]:
class Beholder_Layer_Chopper_Grayscale(Beholder_Layer_Chopper):
    def __init__(self, name, from_layer):
        self.name = name
        self.from_layer = from_layer

    def run(self, bh: Beholder):
        o = {}
        o[self.name] = Beholder_Layer(
            name=self.name,
            data=cv2.cvtColor(bh.layers[self.from_layer].data, cv2.COLOR_BGR2GRAY),
        )
        return o

In [13]:
class Beholder_Layer_Tesseract(Beholder_Layer_Chopper):
    def __init__(self, name, from_layer):
        self.name = name
        self.from_layer = from_layer

    def run(self, bh: Beholder):
        o = {}

In [14]:
class Beholder_Layer_Chopper_AtCord(Beholder_Layer_Chopper):
    def __init__(self, name, from_layer, x, y, h, w, extract_text=False):
        self.name = name
        self.from_layer = from_layer
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.extract_text = extract_text

    def run(self, bh: Beholder):
        o = {}
        o[self.name] = Beholder_Layer(
            name=self.name,
            data=bh.layers[self.from_layer].data[
                self.y : self.y + self.h, self.x : self.x + self.w
            ],
            offsets=(self.x, self.y),
        )
        if self.extract_text:
            img = Image.fromarray(bh[self.from_layer].data)
            img = img.convert("L")  # grayscale
            img = img.filter(ImageFilter.MedianFilter())  # a little blur
            img = img.point(lambda x: 0 if x < 140 else 255)  # threshold (binarize)

            txt = pytesseract.image_to_string(img)
            o[f"{self.name}_Text"] = txt.splitlines()
        return o

In [15]:
a = Beholder(videoFrameGenerator=pullPhoneScreen)

a.addLayerModifer(Beholder_Layer_Chopper_Grayscale(name="gray", from_layer="image"))

a.addLayerModifer(
    Beholder_Layer_Chopper_AtCord(
        name="inventory", from_layer="gray", x=1903, y=355, w=2222 - 1903, h=905 - 355
    )
)

# Below is Auto-Resmith


# Below is Auto Feathers on Bolts

In [16]:
a.readNextImage()
a.digestImage()
for item in a.layers.keys():
    print(item, a.layers[item].data.shape)

image (1080, 2400, 3)
gray (1080, 2400)
inventory (550, 319)


In [17]:
for item in a.matchers:
    print(item.name, item.data.shape)

In [20]:
matches = a.findMatches()
print(matches.keys())

dict_keys(['paper', 'ore', 'orebox'])


In [18]:
# Auto OreChest/NotePaper Mining

In [29]:
a.matchers = {}
a.addDefaultMatcher(
    name="paper", layer="inventory", filename="./templates/items/notepaper.png"
)
a.addDefaultMatcher(
    name="ore", layer="inventory", filename="./templates/ore/addy_inventory.png"
)
a.addDefaultMatcher(
    name="orebox", layer="inventory", filename="./templates/ore/box2.png"
)
a.matchers

{'paper': BW: paper, 'ore': BW: ore, 'orebox': BW: orebox}

In [ ]:
start_mining = 0
ore_box_full = False
while True:
    if time.time() - start_mining > 15:
        print("reset mining")
        adb(f"shell input tap 1215 830")
        start_mining = time.time()
    matches = a.findMatches()
    if len(matches.keys()) > 0:
        if "ore" in matches:
            print(f"{matches['ore'][0][2]} ores in inventory")
            if matches["ore"][0][2] >= 20:
                if not ore_box_full:
                    if "orebox" in matches:
                        print("oreboxing")
                        center = matches["orebox"][0][1]
                        adb(f"shell input tap {center[0]} {center[1]}")
                        matches = a.findMatches()
                        print(matches)
                        if "ore" in matches.keys():
                            ore_box_full = True
                        else:
                            start_mining = 0
                            continue

                elif "ore" in matches:
                    print("papering")
                    center = matches["ore"][0][1]
                    adb(f"shell input tap {center[0]} {center[1]}")
                    time.sleep(0.5)
                    center = matches["paper"][0][1]
                    adb(f"shell input tap {center[0]} {center[1]}")
                    start_mining = 0

    time.sleep(10)

reset mining
reset mining
1 ores in inventory
1 ores in inventory
reset mining
2 ores in inventory
2 ores in inventory
reset mining
3 ores in inventory


In [ ]:
s = time.time()

In [ ]:
time.time() - s

In [ ]:
a.layers["gray"].data.shape